In [1]:
# Install necessary packages
!pip install requests beautifulsoup4 pandas opencv-python

# Import required libraries
import requests
from bs4 import BeautifulSoup
import os
import cv2

# Function to get image URLs from a search URL
def get_image_urls(search_url, max_images=50):
    # Send a GET request to the search URL
    response = requests.get(search_url)
    # Parse the HTML content of the response
    soup = BeautifulSoup(response.text, 'html.parser')
    # Find all 'a' tags with class 'iusc' (these contain image data)
    image_elements = soup.find_all('a', class_='iusc')
    
    urls = []
    # Iterate through the found image elements, up to the specified max_images
    for img in image_elements[:max_images]:
        # Get the 'm' attribute (contains image URL info)
        m = img.get('m')
        if m:
            # Convert the string representation of a dictionary to an actual dictionary
            m = eval(m)
            if 'murl' in m:
                # Append the image URL to the list
                urls.append(m['murl'])
    
    # Return the list of image URLs
    return urls

# Function to download images from a list of URLs
def download_images(urls, folder_name):
    # Create the folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    # Iterate over the list of URLs
    for i, url in enumerate(urls):
        try:
            # Get the content of the image URL
            img_data = requests.get(url).content
            # Save the image data to a file
            with open(f'{folder_name}/{i+1}.jpg', 'wb') as handler:
                handler.write(img_data)
        except Exception as e:
            # Print an error message if the image can't be downloaded
            print(f"Could not download {url}: {e}")

# Define search URLs for different cat breeds
urls = {
    'Maine Coon': 'https://www.bing.com/images/search?q=maine+coon+cat+images&form=HDRSC4&first=1',
    'Persian': 'https://www.bing.com/images/search?q=persian+cat+images&form=HDRSC4&first=1',
    'Siamese': 'https://www.bing.com/images/search?q=siamese+cat+images&FORM=HDRSC4'
}

# Fetch and download images for each cat breed
for cat, url in urls.items():
    # Get image URLs for the specified breed
    image_urls = get_image_urls(url, max_images=50)  # You can increase max_images if needed
    # Download images for the specified breed
    download_images(image_urls, cat)

print("Image downloading complete.")

#Standardize Data
# Function to resize images to a standard size
def resize_images(input_folder, output_folder, size=(128, 128)):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Iterate over the files in the input folder
    for filename in os.listdir(input_folder):
        img_path = os.path.join(input_folder, filename)
        # Read the image
        img = cv2.imread(img_path)
        if img is not None:
            try:
                # Resize the image to the specified size
                resized_img = cv2.resize(img, size)
                # Save the resized image to the output folder
                cv2.imwrite(os.path.join(output_folder, filename), resized_img)
            except Exception as e:
                # Print an error message if the image can't be resized
                print(f"Could not resize {img_path}: {e}")

# Resize images for each cat breed
for cat in urls.keys():
    resize_images(cat, f'resized_{cat}') #creating dataset

print("Image standardization complete.")


Could not download https://www.pentaxforums.com/gallery/images/3123/1_IMGP7088_Thinned_2.jpg: HTTPSConnectionPool(host='www.pentaxforums.com', port=443): Max retries exceeded with url: /gallery/images/3123/1_IMGP7088_Thinned_2.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001AD1541BBB0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))
Image downloading complete.
Image standardization complete.
